In [6]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Amazon
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv, Node2Vec
import numpy as np

def get_data(dataset):
    if dataset in ["Photo", "Computers"]:
        path = osp.join('data', dataset)
        dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())
        dataset.shuffle()
        config = {"l2": 0.0005, "heads": 1}
        return dataset, config
    elif dataset in ["Cora", "Citeseer"]:
        path = osp.join('data', dataset)
        dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
        config = {"l2": 0.0005, "heads": 1}
        return dataset, config
    elif dataset == "Pubmed":
        path = osp.join('data', dataset)
        dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
        config = {"l2": 0.001, "heads": 8}
        return dataset, config
    else:
        print("Unsupported Dataset")

AttributeError: module 'torch' has no attribute '_utils_internal'

In [2]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, config):
        super(Net, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(8 * 8, out_channels, heads=config["heads"], concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)
    
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs

In [3]:
def train_node2vec(data):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                     context_size=10, walks_per_node=10,
                     num_negative_samples=1, p=1, q=1, sparse=True).to(device)

    loader = model.loader(batch_size=128, shuffle=True, num_workers=0)
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

    def train_n2v():
        model.train()
        total_loss = 0
        for pos_rw, neg_rw in loader:
            optimizer.zero_grad()
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(loader)

    @torch.no_grad()
    def test_n2v():
        model.eval()
        z = model()
        acc = model.test(z[data.train_mask], data.y[data.train_mask],
                         z[data.test_mask], data.y[data.test_mask],
                         max_iter=150)
        return acc

    for epoch in range(1, 20):
        loss = train_n2v()
        acc = test_n2v()
        #print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc: {acc:.4f}')
    return model

In [4]:
values = []
for dataset in ["Cora"] * 10:
    dataset, config = get_data(dataset)
    dataset.shuffle()
    TRAIN_END = 20 * dataset.num_classes
    VAL_END = TRAIN_END + 500
    TEST_END = VAL_END + 1000
    data = dataset[0]
    data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
    data.train_mask[:TRAIN_END] = True
    data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
    data.val_mask[TRAIN_END:VAL_END] = True
    data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
    data.test_mask[VAL_END:TEST_END] = True
    
    #data.edge_index = torch.empty((2,0), dtype=torch.long)  # no edges
    
    #data.x = torch.empty((data.num_nodes,data.num_features)) # no features
    
    FIXED_FEAT_VALUE = 1/745
    #data.x = torch.full((data.num_nodes,data.num_features), FIXED_FEAT_VALUE, dtype=torch.float) # fixed features

    #data.x = train_node2vec(data)() #Node2Vec embeddings
    
    #orch.manual_seed(0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data = data.to(device)

    model = Net(data.x.shape[1], dataset.num_classes, config).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=config["l2"])

    print()
    print(dataset)
    max_val = 0
    early_stop_cnt = 0
    for epoch in range(1, 1001):
        train(data)
        train_acc, val_acc, test_acc = test(data)

        if val_acc > max_val:
            early_stop_cnt = 0
            max_val = val_acc
        else:
            early_stop_cnt += 1
            if (early_stop_cnt == 100):
                print(f'Early Stopping at epoch {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
                values = values+[test_acc]
                break



Cora()
Early Stopping at epoch 219, Val: 0.8100, Test: 0.8330

Cora()
Early Stopping at epoch 168, Val: 0.8020, Test: 0.8220

Cora()
Early Stopping at epoch 235, Val: 0.7980, Test: 0.8240

Cora()
Early Stopping at epoch 162, Val: 0.8040, Test: 0.8310

Cora()
Early Stopping at epoch 248, Val: 0.7880, Test: 0.8230

Cora()
Early Stopping at epoch 118, Val: 0.8020, Test: 0.8290

Cora()
Early Stopping at epoch 268, Val: 0.7960, Test: 0.8310

Cora()
Early Stopping at epoch 150, Val: 0.7940, Test: 0.8250

Cora()
Early Stopping at epoch 181, Val: 0.7960, Test: 0.8260

Cora()
Early Stopping at epoch 177, Val: 0.7860, Test: 0.8240


In [5]:
print(np.mean(values))
print(np.std(values))

0.8267999947071075
0.0036823803473353526
